In [1]:
import re
from IPython.display import display, Markdown
from lxml import etree
from os import listdir
from os.path import isfile, join
from tqdm import tqdm

In [2]:
inputFolderPersons = '../data/source/person/'
ns = {"m": "http://www.zetcom.com/ria/ws/module"}

Read input directory

In [3]:
personFiles = [f for f in listdir(inputFolderPersons) if isfile(join(inputFolderPersons, f)) and f.endswith('.xml')]

Set limit of number of files to process (only used for debugging)

In [4]:
limit = 99999

Read input files and retrieve required data

In [5]:
personItems = {}
for file in tqdm(personFiles[:limit]):
    tree = etree.parse(join(inputFolderPersons, file))
    uuid = tree.find('.//m:moduleItem', ns).get('uuid')
    fields = tree.findall('.//m:moduleItem/m:repeatableGroup[@name="PerURLGrp"]/m:repeatableGroupItem', ns)
    if not uuid in personItems:
        personItems[uuid] = {}
    if len(fields) > 0:
        for field in fields:
            try:
                vocab = field.find('m:vocabularyReference[@name="TypeVoc"]/m:vocabularyReferenceItem/m:formattedValue', ns).text
            except:
                vocab = 'undefined'
            try:
                value = field.find('m:dataField[@name="AddressTxt"]/m:value', ns).text
            except:
                value = 'undefined'
            if not vocab in field:
                personItems[uuid][vocab] = []
            personItems[uuid][vocab].append(value)

100%|██████████| 2455/2455 [00:00<00:00, 5416.21it/s]


Collect all URLs organised by type

In [6]:
values = {}
for item in personItems.values():
    for key in item.keys():
        if not key in values:
            values[key] = []
        for url in item[key]:
            values[key] += item[key]

In [7]:
personItems

{'1734': {},
 'eac5d950-411a-43ae-b1b1-5ae9b1b6ba26': {},
 '687': {},
 '1052': {},
 '1046': {},
 '693': {},
 'b69875b7-ead0-46f0-8a95-1df78ba3a3f1': {},
 'dc98c7b1-7389-478d-8039-414ef9edab73': {},
 '1708': {},
 '877': {},
 'f6462810-20af-46f2-96a8-d3e124689d26': {},
 '122': {},
 '1091': {'Website (externe Referenz)': ['https://hls-dhs-dss.ch/de/articles/048790/2006-12-12/']},
 '888': {},
 '650': {},
 '1085': {},
 '1419ee74-37de-4ab3-af98-2b46d25e8f74': {},
 '678': {},
 '6d52c950-ad99-4382-82bd-7b8d9b0606b4': {},
 '00ccf318-91ce-4e85-9894-a3536d20a3d7': {},
 '1907': {},
 '8f17d661-56c8-4d7c-b1ec-d55f1f25caf6': {},
 '1250': {},
 'dde591c7-be97-4f42-b663-40ecb956c00e': {},
 '1536': {},
 '7468063b-70e0-4c2e-a28d-3b1713ee1c65': {},
 '2fa62f4d-6d95-4c12-b016-5cebbce049b0': {},
 '00a7f209-b719-494b-ad67-c45786db4a34': {},
 '2003': {'undefined': ['http://www.winzeler.ch/de/company_history.php']},
 '35f5d6b9-340e-4a53-b899-ee7a438baefd': {},
 '2017': {},
 '1278': {},
 '446': {},
 '1293': {},
 

In [8]:
normalisePattern = re.compile(r'\d{4,}')
patterns = {}
for source, urls in values.items():
    if source not in patterns:
        patterns[source] = {
            "patterns": set(),
            "examples": {}
        }
    for url in urls:
        normalisedUrl = normalisePattern.sub('#️⃣', url)
        patterns[source]["patterns"].add(normalisedUrl)
        patterns[source]["examples"][normalisedUrl] = url

In [15]:
output = ''
for source in sorted(patterns.keys()):
    output += f"## Context: {source}\n\nPatterns:\n\n"
    for pattern in sorted(patterns[source]["patterns"]):
        output += f"- {pattern} \n"
        if patterns[source]['examples'][pattern] != pattern:
            output+= f"  - e.g. {patterns[source]['examples'][pattern]}\n\n"

In [16]:
display(Markdown(output))

## Context: BnF

Patterns:

- http://ark.bnf.fr/ark:/#️⃣/cb#️⃣ 
  - e.g. http://ark.bnf.fr/ark:/12148/cb122565479

- http://ark.bnf.fr/ark:/#️⃣/cb#️⃣d 
  - e.g. http://ark.bnf.fr/ark:/12148/cb16700755d

- http://ark.bnf.fr/ark:/#️⃣/cb#️⃣q 
  - e.g. http://ark.bnf.fr/ark:/12148/cb11881358q

- http://ark.bnf.fr/ark:/#️⃣/cb#️⃣v 
  - e.g. http://ark.bnf.fr/ark:/12148/cb13574024v

## Context: RKD

Patterns:

-  https://rkd.nl/explore/artists/#️⃣ 
  - e.g.  https://rkd.nl/explore/artists/3631

- https://rkd.nl/en/explore/artists/#️⃣ 
  - e.g. https://rkd.nl/en/explore/artists/88417

- https://rkd.nl/explore/artists/#️⃣ 
  - e.g. https://rkd.nl/explore/artists/115000

## Context: SIK-ISEA

Patterns:

- #️⃣ 
  - e.g. 4025024

- http://www.sikart.ch/KuenstlerInnen,aspx?id=#️⃣&lng=de 
  - e.g. http://www.sikart.ch/KuenstlerInnen,aspx?id=4024619&lng=de

- http://www.sikart.ch/KuenstlerInnen.aspx?id=#️⃣&lng=de 
  - e.g. http://www.sikart.ch/KuenstlerInnen.aspx?id=4025091&lng=de

- http://www.sikart.ch/KuenstlerInnen.aspx?id=#️⃣&lng=en 
  - e.g. http://www.sikart.ch/KuenstlerInnen.aspx?id=4024531&lng=en

- http://www.sikart.ch/KuenstlerInnen.aspx?id=#️⃣&lng=fr 
  - e.g. http://www.sikart.ch/KuenstlerInnen.aspx?id=4023674&lng=fr

- http://www.sikart.ch/kuenstlerinnen.aspx?id=#️⃣ 
  - e.g. http://www.sikart.ch/kuenstlerinnen.aspx?id=4023097

- https://recherche.sik-isea.ch/de/sik:person-#️⃣/in/sikisea/actor/list?0.0.type=actor&0.0.@id=sik:person-#️⃣&0.type=actor 
  - e.g. https://recherche.sik-isea.ch/de/sik:person-4003803/in/sikisea/actor/list?0.0.type=actor&0.0.@id=sik:person-4003803&0.type=actor

- https://recherche.sik-isea.ch/sik:person-#️⃣ 
  - e.g. https://recherche.sik-isea.ch/sik:person-4002003

- https://recherche.sik-isea.ch/sik:person-#️⃣/in/sikart 
  - e.g. https://recherche.sik-isea.ch/sik:person-4022925/in/sikart

- https://recherche.sik-isea.ch/sik:person-#️⃣/in/sikisea 
  - e.g. https://recherche.sik-isea.ch/sik:person-4000714/in/sikisea

- https://www.sikart.ch/KuenstlerInnen.aspx?id=#️⃣ 
  - e.g. https://www.sikart.ch/KuenstlerInnen.aspx?id=4002964

- https://www.sikart.ch/kuenstlerInnen.aspx?id=#️⃣ 
  - e.g. https://www.sikart.ch/kuenstlerInnen.aspx?id=4000737

- https://www.sikart.ch/kuenstlerinnen.aspx?id=#️⃣ 
  - e.g. https://www.sikart.ch/kuenstlerinnen.aspx?id=4025495

## Context: SIKART ID

Patterns:

- #️⃣ 
  - e.g. 4028863

- sik:person-#️⃣ 
  - e.g. sik:person-4025015

- sik:person-#️⃣:exp 
  - e.g. sik:person-4003647:exp

## Context: Sonstige Referenz

Patterns:

- https://artuk.org/discover/artists/prower-dora-#️⃣ 
  - e.g. https://artuk.org/discover/artists/prower-dora-19071996

- https://collections.geneve.ch/mah/auteur/fonderie-dart-mario-pastori 
- https://rkd.nl/explore/artists/#️⃣ 
  - e.g. https://rkd.nl/explore/artists/212605

- https://www.diju.ch/f/notices/detail/#️⃣ 
  - e.g. https://www.diju.ch/f/notices/detail/5061

- https://www.porcelainmarksandmore.com/germany/thuringia/katzhuette-01/index.php 
## Context: Website (eigene)

Patterns:

- http://www.heinz-keller.ch 
- http://www.sikart.ch/KuenstlerInnen.aspx?id=#️⃣&lng=de 
  - e.g. http://www.sikart.ch/KuenstlerInnen.aspx?id=4005153&lng=de

- http://www.siskov.at 
- https://faksimile.de 
- https://glassart-tobler.ch 
- https://hanneke-beaumont.com/ 
- https://majolika-karlsruhe.de/ 
- https://steinlin.com 
- https://valim-art.ch/en/index.html 
- https://www.babyinktwice.ch/ 
- https://www.kunstgesellschaft.ch/ 
- https://www.luisadelcampana.com 
## Context: Website (externe Referenz)

Patterns:

- http://d-nb.info/gmd/#️⃣ 
  - e.g. http://d-nb.info/gmd/118577891

- http://fondazionegiudicetti.ch 
- http://kunstsammlung.peterschmelzle.de 
- http://sikart.ch/KuenstlerInnen.aspx?id=#️⃣&lng=de 
  - e.g. http://sikart.ch/KuenstlerInnen.aspx?id=11027724&lng=de

- http://sikart.ch/Kuenstlerinnen.aspx?id=#️⃣&lng=de 
  - e.g. http://sikart.ch/Kuenstlerinnen.aspx?id=4002518&lng=de

- http://www.albert-gieseler.de/dampf_de/firmen1/firmadet#️⃣.shtml 
  - e.g. http://www.albert-gieseler.de/dampf_de/firmen1/firmadet10142.shtml

- http://www.artnet.com/artists/henry-wilkinson/ 
- http://www.familie-untersteller.de 
- http://www.sikart.ch.KunstleriInnenaspx?id=#️⃣&Ing=de 
  - e.g. http://www.sikart.ch.KunstleriInnenaspx?id=4023599&Ing=de

- http://www.sikart.ch/KuenstlerInnen,aspx?id=#️⃣&lng=de 
  - e.g. http://www.sikart.ch/KuenstlerInnen,aspx?id=4025843&lng=de

- http://www.sikart.ch/KuenstlerInnen.aspx?id=#️⃣&lng=de 
  - e.g. http://www.sikart.ch/KuenstlerInnen.aspx?id=4025370&lng=de

- http://www.sikart.ch/KuenstlerInnen.aspx?id=#️⃣=de 
  - e.g. http://www.sikart.ch/KuenstlerInnen.aspx?id=4023451=de

- http://www.sikart.ch/KuenstleriInnen.aspx?id=#️⃣&lng=de 
  - e.g. http://www.sikart.ch/KuenstleriInnen.aspx?id=4026519&lng=de

- http://www.sikart.ch/Kuenstlerinnen.aspx?id=#️⃣&lng=de 
  - e.g. http://www.sikart.ch/Kuenstlerinnen.aspx?id=403104&lng=de

- http://www.sikart.ch/Kuenstlerinnen.aspx?id=#️⃣lng=de 
  - e.g. http://www.sikart.ch/Kuenstlerinnen.aspx?id=4002499lng=de

- http://www.sikartch/KuenstlerInnen.aspx?id=#️⃣&lng=de 
  - e.g. http://www.sikartch/KuenstlerInnen.aspx?id=4002112&lng=de

- http;//www.sikart.ch/KuenstlerInnen.aspx?id=#️⃣&lng=de 
  - e.g. http;//www.sikart.ch/KuenstlerInnen.aspx?id=4000004&lng=de

- https://bge-geneve.ch 
- https://changestheworld.com/ 
- https://collections.genève.ch 
- https://de.wikipedia.org/wiki/Ad%C3%A8le_Kindt 
- https://de.wikipedia.org/wiki/Affortunato_Gori 
- https://de.wikipedia.org/wiki/Alo%C3%AFse_Corbaz 
- https://de.wikipedia.org/wiki/Anna_Indermaur 
- https://de.wikipedia.org/wiki/Anny_Boxler 
- https://de.wikipedia.org/wiki/Edmund_Wunderlich 
- https://de.wikipedia.org/wiki/Emmy_Glintzer 
- https://de.wikipedia.org/wiki/Eva_Braun 
- https://de.wikipedia.org/wiki/Fabrique_Nationale_Herstal 
- https://de.wikipedia.org/wiki/Fayence-Manufaktur_Frisching 
- https://de.wikipedia.org/wiki/Ginevra_Cantofoli 
- https://de.wikipedia.org/wiki/J%C3%BCrg_Kreienb%C3%BChl 
- https://de.wikipedia.org/wiki/K%C3%A4the_Kollwitz 
- https://de.wikipedia.org/wiki/Klauber_(Kupferstecher) 
- https://de.wikipedia.org/wiki/Kunst-_und_Glockengie%C3%9Ferei_Lauchhammer 
- https://de.wikipedia.org/wiki/Marguerite_Burnat-Provins 
- https://de.wikipedia.org/wiki/Mariette_Lydis 
- https://de.wikipedia.org/wiki/Maxime_Maufra 
- https://de.wikipedia.org/wiki/Raymonde_Guerbe 
- https://de.wikipedia.org/wiki/Rosalba_Carriera 
- https://de.wikipedia.org/wiki/Wehrmacht 
- https://doi.org.10.#️⃣/e-rara-#️⃣ 
  - e.g. https://doi.org.10.3931/e-rara-51943

- https://en.wikipedia.org/wiki/Hanneke_Beaumont 
- https://fr.wikipedia.org/wiki/Fonderie_Clementi 
- https://fr.wikipedia.org/wiki/Fonderie_Rudier 
- https://fr.wikipedia.org/wiki/Jacqueline_Benoit 
- https://hls-dhs-dss.ch/de/articles/#️⃣/#️⃣-02-15 
  - e.g. https://hls-dhs-dss.ch/de/articles/044390/2012-02-15

- https://hls-dhs-dss.ch/de/articles/#️⃣/#️⃣-12-12/ 
  - e.g. https://hls-dhs-dss.ch/de/articles/048790/2006-12-12/

- https://recherche.sik-isea.ch/en/sik:person-#️⃣/in/sikart/actor/list 
  - e.g. https://recherche.sik-isea.ch/en/sik:person-4028863/in/sikart/actor/list

- https://recherche.sik-isea.ch/en/sik:person-#️⃣:exp/in/sikart/actor/list?0.0.type=actor&0.0.@id=sik:person-#️⃣&0.type=actor 
  - e.g. https://recherche.sik-isea.ch/en/sik:person-4022874:exp/in/sikart/actor/list?0.0.type=actor&0.0.@id=sik:person-4022874&0.type=actor

- https://rkd.nl/explore/artists/#️⃣  
  - e.g. https://rkd.nl/explore/artists/5481 

- https://rkd.nl/nl/explore/artists/#️⃣ 
  - e.g. https://rkd.nl/nl/explore/artists/37034

- https://sammlung.belvedere.at 
- https://venderequadri.it/artist-rating/del-campana-luisa/ 
- https://www.biographien.ac.at/oebl/oebl_M/Maly_Paula_#️⃣_#️⃣.xml 
  - e.g. https://www.biographien.ac.at/oebl/oebl_M/Maly_Paula_1891_1974.xml

- https://www.hans-kloss.de/texts/maria-kloss.htm 
- https://www.kunstbreite.ch/Kuesntlerwerdegaenge_aargau_aerni_walter.htm 
- https://www.portraitarchiv.ch/portrait/show/#️⃣ 
  - e.g. https://www.portraitarchiv.ch/portrait/show/274517

- https://www.vontobel-art.com/de/home 
- www.museum-holzbildhauerei.ch 
## Context: Wikidata Identifier

Patterns:

- #️⃣ 
  - e.g. 106832605

- Q#️⃣ 
  - e.g. Q47160232

## Context: undefined

Patterns:

- http://vocab.getty.edu/page/ulan/#️⃣ 
  - e.g. http://vocab.getty.edu/page/ulan/500055302

- http://web.tiscali.it/luisa_del_campana/index.html 
- http://werkverzeichnis.bossard.de/ 
- http://www.basler-kopp.ch/ 
- http://www.dorfmuseum-langnau-mehlsecken.ch 
- http://www.sikart.ch/KuenstleriInnen.aspx?id=#️⃣&lng=de 
  - e.g. http://www.sikart.ch/KuenstleriInnen.aspx?id=4031064&lng=de

- http://www.stauffer-bern.ch/ 
- http://www.winzeler.ch/de/company_history.php 
- http://www.wulz-art.net/ 
- https://de.wikipedia.org/wiki/Christian_Speyer 
- https://de.wikipedia.org/wiki/Elsa_Pletscher 
- https://de.wikipedia.org/wiki/Ir%C3%A8ne_Zurkinden 
- https://de.wikipedia.org/wiki/Pierre_de_Salis-Soglio 
- https://recherche.sik-isea.ch/sik:person-#️⃣/in/sikisea 
  - e.g. https://recherche.sik-isea.ch/sik:person-4031579/in/sikisea

- https://www.britishmuseum.org/collection/term/BIOG#️⃣ 
  - e.g. https://www.britishmuseum.org/collection/term/BIOG19947

- undefined 
